<a href="https://colab.research.google.com/github/yuuki-kusumoto/kusumoto/blob/master/multi_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
!apt install aptitude swig

In [ ]:
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y

In [ ]:
pip install MeCab

In [ ]:
pip install mecab-python3

In [ ]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git

In [ ]:
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

In [ ]:
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')

In [ ]:
pip install neologdn

In [ ]:
pip install ipadic

In [ ]:
pip install mecab-python3

In [ ]:
pip install unidic-lite